In [6]:
import torch
import tensorly as tl
from tensorly.decomposition import partial_tucker
from tltorch import FactorizedConv
import tltorch
from torchvision.models import resnet18
import numpy as np

In [3]:
tl.set_backend('pytorch')

In [2]:


def count_params(net: torch.nn.Module) -> np.array:
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

In [17]:
model = resnet18(weights=None)

In [20]:
n_param = count_params(model)

In [59]:
import copy

factorization = 'tucker'
rank = 0.8
decompose_weights = True
td_init = not decompose_weights

decomposition_kwargs = {'init': 'random'} if factorization == 'cp' else {}
fixed_rank_modes = 'spatial' if factorization == 'tucker' else None

fact_model = copy.deepcopy(model)

ranks = {
    'layer1.0.conv1': (64, 16),
    'layer1.0.conv2': (64, 16),
    'layer1.1.conv1': (64, 16),
    'layer1.1.conv2': (64, 16),
    'layer2.0.conv1': (128, 16),
    'layer2.0.conv2': (128, 16),
    'layer2.1.conv1': (128, 16),
    'layer2.1.conv2': (128, 16),
    'layer3.0.conv1': (256, 16),
    'layer3.0.conv2': (256, 16),
    'layer3.1.conv1': (256, 16),
    'layer3.1.conv2': (256, 16),
    'layer4.0.conv1': (512, 16),
    'layer4.0.conv2': (512, 16),
    'layer4.1.conv1': (512, 16),
    'layer4.1.conv2': (512, 16),
}

layer_names = list(ranks.keys())

for i, (name, module) in enumerate(model.named_modules()):
    if name in layer_names:
        print(f'factorizing: {name}')
        if type(module) == torch.nn.modules.conv.Conv2d:
            fact_layer = tltorch.FactorizedConv.from_conv(
                module, 
                rank=rank, 
                decompose_weights=decompose_weights, 
                factorization=factorization,
                fixed_rank_modes=fixed_rank_modes,
                decomposition_kwargs=decomposition_kwargs,
            )
            if td_init:
                fact_layer.weight.normal_(0, td_init)
            layer, block, conv = name.split('.')
            conv_to_replace = getattr(getattr(fact_model, layer), block)
            setattr(conv_to_replace, conv, fact_layer)
            
n_param_fact = count_params(fact_model)

factorizing: layer1.0.conv1
factorizing: layer1.0.conv2
factorizing: layer1.1.conv1
factorizing: layer1.1.conv2
factorizing: layer2.0.conv1
factorizing: layer2.0.conv2
factorizing: layer2.1.conv1
factorizing: layer2.1.conv2
factorizing: layer3.0.conv1
factorizing: layer3.0.conv2
factorizing: layer3.1.conv1
factorizing: layer3.1.conv2
factorizing: layer4.0.conv1
factorizing: layer4.0.conv2
factorizing: layer4.1.conv1
factorizing: layer4.1.conv2


In [60]:
print(f'original number of parameters: {n_param}')
print(f'factorized number of parameters: {n_param_fact}')
print(f'before - after: {n_param - n_param_fact}')
print(f'compression ratio: {n_param / n_param_fact:.2f}')

original number of parameters: 11689512
factorized number of parameters: 9501963
before - after: 2187549
compression ratio: 1.23


In [61]:
fact_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(54, 54, 3, 3), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(54, 54, 3, 3))
      )
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): FactorizedConv(
        in_channels=64, out_channels=64, kernel_size=(3, 3), rank=(51, 51, 3, 3), order=2, padding=[1, 1], bias=False
        (weight): TuckerTensor(shape=(64, 64, 3, 3), rank=(51, 51, 3, 3))
      )
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True